In [4]:
import spacy
import pandas as pd

nlp = spacy.load("en_core_web_sm")

In [5]:
df = pd.read_csv("pronoun_testcases.csv")

In [6]:
def transform_with_spacy(text, target_gender):
    doc = nlp(text)
    transformed = []

    for token in doc:
        orig = token.text
        lower = token.text.lower()
        tag = token.tag_  # e.g., PRP, PRP$

        new_word = orig

        if target_gender == "female":
            if lower == "he":
                new_word = "she"
            elif lower == "him":
                new_word = "her"
            elif lower == "his":
                new_word = "her" if tag == "PRP$" else "his"
            elif lower == "himself":
                new_word = "herself"

        elif target_gender == "male":
            if lower == "she":
                new_word = "he"
            elif lower == "her":
                new_word = "his" if tag == "PRP$" else "him"
            elif lower == "hers":
                new_word = "his"
            elif lower == "herself":
                new_word = "himself"

        if orig[0].isupper():
            new_word = new_word.capitalize()

        transformed.append((new_word, token.whitespace_))

    return ''.join([w + ws for w, ws in transformed])

df['predicted_output'] = df.apply(
    lambda row: transform_with_spacy(row['input_text'], row['target_gender']),
    axis=1
)

In [7]:
df['correct'] = df['predicted_output'] == df['expected_output']
correct = df['correct'].sum()
total = len(df)
accuracy = correct / total * 100

print(f"\n Accuracy: {correct}/{total} correct ({accuracy:.2f}%)")


 Accuracy: 26/26 correct (100.00%)
